<a href="https://colab.research.google.com/github/distinct93/Market-basket-Analysis/blob/main/Emmanuel_N_webscraping_stackexchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

pip install requests_html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 15.3 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=5f60dda8d8884558c1533c5fd75f491982c6109d20f60aeede213854608818bf
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [ ]:
html = HTML(html = html_str)

In [ ]:
classes_needed = ['.s-post-summary--stats-item-number',  '.s-post-summary--content-title', '.s-post-summary--content-excerpt',]

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def build_url(sort_by, page_number):
    sort_by='votes&page='
    base_url = "https://stackoverflow.com/questions?tab="
    return f'{base_url}{sort_by}{page_number}'

def fetch_page(url):
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")

def extract_questions(soup):
    classes_needed = ['.s-post-summary--stats-item-number', '.s-post-summary--content-title', '.s-post-summary--content-excerpt', '.post-tag']
    key_names = ['votes', 'summary', 'question', 'tags']
    question_classes = soup.select('.s-post-summary')
    datas = []

    for question in question_classes:
        question_data = {}
        for i, _class in enumerate(classes_needed):
            sub_element = question.select_one(_class)
            keyname = key_names[i]
            question_data[keyname] = sub_element.text if sub_element else None
        datas.append(question_data)

    return datas

def scrape_stack_overflow(sort_by='votes&page=', start_page=1, end_page=50):
    all_data = []
    for page_number in range(start_page, end_page + 1):
        print(f"Scraping page {page_number}...")
        url = build_url(sort_by, page_number)
        soup = fetch_page(url)
        data = extract_questions(soup)
        all_data.extend(data)
    return pd.DataFrame(all_data)

if __name__ == "__main__":

    start_page = 1
    end_page = 10
    questions_df = scrape_stack_overflow(start_page=start_page, end_page=end_page)
    print(questions_df)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
     votes                                            summary  \
0    27128  \nWhy is processing a sorted array faster than...   
1    25978  \nHow do I undo the most recent local commits ...   
2    20334  \nHow do I delete a Git branch locally and rem...   
3    13708  \nWhat is the difference between 'git pull' an...   
4    12761    \nWhat does the "yield" keyword do in Python?\n   
..     ...                                                ...   
495   2112  \nHow can I update Node.js and NPM to their la...   
496   2111         \nPreview an image before it is uploaded\n   
497   2109               \nDownload a specific tag with Git\n   
498   2104  \nHow to replace master branch in Git, entirel...   
499   2103           \nHow to convert int to string in C++?\n   

                            

In [2]:
questions_df.head()

,votes,summary,question,tags
0,27128,\nWhy is processing a sorted array faster than...,"\r\n In this C++ code, sorting ...",java
1,25978,\nHow do I undo the most recent local commits ...,\r\n I accidentally committed t...,git
2,20334,\nHow do I delete a Git branch locally and rem...,\r\n Failed Attempts to Delete ...,git
3,13708,\nWhat is the difference between 'git pull' an...,\r\n What are the differences b...,git
4,12761,"\nWhat does the ""yield"" keyword do in Python?\n",\r\n What is the use of the yie...,python


In [3]:
def clean_text(df):
    # Remove \r\n from the question column
    df['question'] = df['question'].apply(lambda x: x.replace('\r\n', ' ') if x else x)

    # Remove \n from the summary column
    df['summary'] = df['summary'].apply(lambda x: x.replace('\n', ' ') if x else x)

    return df

    if __name__ == "__main__":
      tag = "python"
      start_page = 1
      end_page = 5
      questions_df = scrape_stack_overflow(tag, start_page=start_page, end_page=end_page)

      # Clean up the text in the question and summary columns
      questions_df = clean_text(questions_df)

    print(questions_df)


In [4]:
clean_df = clean_text(questions_df)

In [5]:
clean_df.to_parquet('full_stack_questions.parquet', index=False)